In [1]:
import os
os.environ['YT_DEST'] = '/home/kwoksun2/anaconda2/pkgs/yt-3.3.5-np111py27_2/'
import numpy as np
from dengo.chemical_network import \
    ChemicalNetwork, \
    reaction_registry, \
    cooling_registry, species_registry
import dengo.primordial_rates, dengo.primordial_cooling
from dengo.chemistry_constants import tiny, kboltz, mh
import yt
import yt.units as u
import numpy
import pickle
import pyximport
import h5py
import timeit
import time
from generate_dls_cooling_network import create_cvdls_solver, Init_values
import matplotlib.pyplot as plt

 using cli
 using CLI for selections


In [2]:
import os, sys

class HidePrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout

# Evolve Free Fall Parcel

1. let the gas cool at constant density from the starting temperature down to a lower temperature to get the species fractions in a resonable state
2. evolve density and temperature according to freefall collapse

In [3]:
init, primordial = Init_values(np.array([2000.0]), np.array([1e10]) , n_species = 9, cooling=True)

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2
Adding reaction: k07 : 1*H_1 + 1*de => 1*H_m0
Adding reaction: k08 : 1*H_m0 + 1*H_1 => 1*H2_1 + 1*de
Adding reaction: k09 : 1*H_1 + 1*H_2 => 1*H2_2
Adding reaction: k10 : 1*H2_2 + 1*H_1 => 1*H2_1 + 1*H_2
Adding reaction: k11 : 1*H2_1 + 1*H_2 => 1*H2_2 + 1*H_1
Adding reaction: k12 : 1*H2_1 + 1*de => 2*H_1 + 1*de
Adding reaction: k13 : 1*H2_1 + 1*H_1 => 3*H_1
Adding reaction: k14 : 1*H_m0 + 1*de => 1*H_1 + 2*de
Adding reaction: k15 : 1*H_m0 + 1*H_1 => 2*H_1 + 1*de
Adding reaction: k16 : 1*H_m0 + 1*H_2 => 2*H_1
Adding reaction: k17 : 1*H_m0 + 1*H_2 => 1*H2_2 + 1*de
Adding reaction: k18 : 1*H2_2 + 1*de => 2*H_1
Adding reaction: k19 : 1*H2_2 + 1*H_m0 => 1*H_1 + 1*H2_1
Adding reaction: k21 : 2*H_1 

In [4]:
primordial.cooling_actions

{'brem': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0610>,
 'ceHI': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0290>,
 'ceHeI': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0390>,
 'ceHeII': <dengo.reaction_classes.CoolingAction at 0x7f6486bc03d0>,
 'ciHI': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0450>,
 'ciHeI': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0490>,
 'ciHeII': <dengo.reaction_classes.CoolingAction at 0x7f6486bc04d0>,
 'ciHeIS': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0410>,
 'cie_cooling': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0750>,
 'compton': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0690>,
 'gammah': <dengo.reaction_classes.CoolingAction at 0x7f6486bc06d0>,
 'gloverabel08': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0650>,
 'h2formation': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0710>,
 'reHII': <dengo.reaction_classes.CoolingAction at 0x7f6486bc0510>,
 'reHeII1': <dengo.reactio

In [5]:
def calculate_pressure(init, primordial):
    P = numpy.zeros((1))
    T = init['T']
    for sp in primordial.required_species:
        if sp.name != 'ge':
            n_sp = init[sp.name]/sp.weight
            P += n_sp * u.boltzmann_constant_cgs.v * T
    return P

def calculate_collapse_factor(pressure, density):
    # Calculate the effective adiabatic index, dlog(p)/dlog(rho).
    if len(pressure) < 3:
        return 0.

    # compute dlog(p) / dlog(rho) using last two timesteps
    gamma_eff = np.log10(pressure[-1] / pressure[-2]) / \
        np.log10(density[-1] / density[-2])
    
    # compute a higher order derivative if more than two points available
    if len(pressure) > 2:
        gamma_eff += 0.5 * ((np.log10(pressure[-2] / pressure[-3]) /
                             np.log10(density[-2] / density[-3])) - gamma_eff)

    gamma_eff = min(gamma_eff, 4./3.)

    # Equation 9 of Omukai et al. (2005)
    if gamma_eff < 0.83:
        force_factor = 0.0
    elif gamma_eff < 1.0:
        force_factor = 0.6 + 2.5 * (gamma_eff - 1) - \
            6.0 * np.power((gamma_eff - 1.0), 2.)
    else:
        force_factor = 1.0 + 0.2 * (gamma_eff - (4./3.)) - \
            2.9 * np.power((gamma_eff - (4./3.)), 2.)

    force_factor = max(force_factor, 0.0)
    force_factor = min(force_factor, 0.95)
    return force_factor

def calculate_gamma(init, primordial):
    gamma = 5.0/3.0
    for sp in primordial.required_species: 
        if sp.name == 'H2_1':
            sp_H2 = sp
            break
    gammaH2 = primordial.species_gamma(sp, temp=True, name=False).subs({'T':temperature})

    gamma_fac = primordial.gamma_factor()
    gamma_factor = gamma_fac.subs(init).subs({'gamma':gamma}).subs({'gammaH2':gammaH2})

    n_density = 0.0
    for sp in primordial.required_species:
        if sp.name != 'ge':
            n_density += init[sp.name]

    gamma_ad = n_density/gamma_factor + 1
    print(gamma_ad)
    gamma_ad = float(gamma_ad)
    return gamma_ad



def calculate_temperature(init, primordial):
    dT = 10.0
    temperature = init['T']
        
    while dT > 0.1:
        x = 6100.0/temperature
        # update the gammaH2 which is dependent on temperature
        gammaH2 = 2.0 / (5.0 + 2.0*x*x*numpy.exp(x) / (numpy.exp(x) - 1 )**2.0 ) + 1

        gamma_factor = primordial.gamma_factor().subs(init).subs({'gammaH2': gammaH2 , 'gamma': 5./3.,'T':temperature })
        
        # with ge updated from compressional heating
        ge = init['ge']
        
        new_T = numpy.array([float(init['density']*ge*mh / kboltz / gamma_factor)])
        dT = numpy.abs(new_T - temperature)
        temperature = new_T
        
    return new_T

def calculate_energy(init, primordial):
    """Calculate energy from the abundance and temperature
    """
    num_den = {}
    for sp in primordial.required_species:
        try:
            num_den[sp.name] = init[sp.name]/ sp.weight
        except:
            pass

    # set up initial temperatures values used to define ge
    temperature = init['T']

    # calculate gammaH2
    x = 6100.0/temperature
    gammaH2 = 2.0 / (5.0 + 2.0*x*x*numpy.exp(x) / (numpy.exp(x) - 1 )**2.0 ) + 1

    gamma_factor = primordial.gamma_factor().subs(num_den).subs({'gammaH2': gammaH2 , 'gamma': 5./3.,'T': temperature })

    ge  = ((temperature *  kboltz) *gamma_factor
                         / (init['density'] * mh  ))

    T = init['density']*ge*mh / kboltz / gamma_factor
    
    print(T-temperature)
    
    return numpy.array( [numpy.float64(ge)] )


In [6]:
def update_initial_condition(init, primordial, pressure_array, density_array, safety_factor=0.01):
    
    # should be in cgs units
    # dyne / cm^-2
    current_pressure = calculate_pressure(init, primordial)
    pressure_array = numpy.append(pressure_array, current_pressure)    
    
    
    include_pressure = False
    if include_pressure:
        force_factor = calculate_collapse_factor(pressure_array, density_array)
    else:
        force_factor = 0.0    
    print("force_factor: {}".format(force_factor))
    
    density = init['density']
    
    # compute the new density using the modified
    # free-fall collapse as per Omukai et al. (2005)

    gravitational_constant = 4.0*numpy.pi*6.65259e-8 *  u.amu_cgs.v
    freefall_time_constant = np.power((( 32.0*gravitational_constant)/ (3.0*numpy.pi)), 0.5)

    dt = safety_factor* np.power( (3.0*np.pi)/ (32.0* gravitational_constant *density ), 0.5 )

    # calculate new density from altered free-fall solution
    
    new_density = np.power((np.power(density, -0.5) -
                                (0.5 * freefall_time_constant * dt *
                                 np.power((1 - force_factor), 0.5))), -2.)
    
    # multiply this with the elemental abundances
    density_ratio = new_density/density

    # update densities
    # only update the species array only
    for sp in primordial.required_species:
        if sp.name != 'ge':
            init[sp.name] *= density_ratio

    Gamma = calculate_gamma(init, primordial)
    
    # update internal energy
    init['ge'] += (Gamma - 1.0) * init['ge'] * \
                        freefall_time_constant* \
                        new_density**0.5 * dt
            
    print( "gammma - 1: {}".format((Gamma - 1.0)))
    # update density
    init['density'] = new_density
    density_array = numpy.append(density_array, new_density)
    
    
    # update temperature with the updated internal energy
    init['T'] = calculate_temperature(init, primordial)
    
    return init, pressure_array, density_array, dt, force_factor


def generate_init_from_results(rv_int, primordial, old_init):
    flag = rv_int['successful']
    init = {}
    for sp in primordial.required_species:
        print(sp)
        init[sp.name] = rv_int[sp.name][0][flag][-1]*sp.weight
    density = old_init['density']
    init['density'] = density
    init['T'] = numpy.array([rv_int['T'][0][flag][-1]])
    return init


def convert_from_grackle_to_dengo(grackle_dict):
    dengo_dict = {}
    for key in grackle_dict:
        key = str(key)
        
        ele = key.split('I')[0]
        charge = key.count('I')
        if charge > 0:
            dengo_name = ele+ '_' + str(charge)
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key][0])/u.amu_cgs.v
        elif 'M' in key:
            ele = key.split('M')[0]
            dengo_name = ele + '_' + str("m0")
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key][0])/u.amu_cgs.v
        elif key == 'temperature':
            dengo_name = 'T'
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key][0])
        elif key == 'de':
            dengo_name = 'de'
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key][0])/u.amu_cgs.v
    return dengo_dict


def convert_from_grackle_to_dengo_all(grackle_dict):
    dengo_dict = {}
    for key in grackle_dict:
        key = str(key)
        
        ele = key.split('I')[0]
        charge = key.count('I')
        if charge > 0:
            dengo_name = ele+ '_' + str(charge)
            if ele == 'H':
                dengo_dict[dengo_name] = numpy.array(grackle_dict[key])/u.amu_cgs.v / 1.00794
            elif ele == 'He':
                dengo_dict[dengo_name] = numpy.array(grackle_dict[key])/u.amu_cgs.v / 4.002602
            elif ele == 'H2':
                dengo_dict[dengo_name] = numpy.array(grackle_dict[key])/u.amu_cgs.v / 1.00794 /2.0
        elif 'M' in key:
            ele = key.split('M')[0]
            dengo_name = ele + '_' + str("m0")
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key])/u.amu_cgs.v
        elif key == 'temperature':
            dengo_name = 'T'
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key])
        elif key == 'de':
            dengo_name = 'de'
            dengo_dict[dengo_name] = numpy.array(grackle_dict[key])/u.amu_cgs.v
    return dengo_dict

In [8]:
# Initial conditions
temperature = 20000.0# K
density = 1.0e-1 # cm^-3

solver_name = 'cvdls_9species'
with HidePrints():
    init, primordial = Init_values(np.array([temperature]), np.array([density]) , n_species = 9
                                   , cooling=True)

chemistry_run = create_cvdls_solver(init,primordial, solver_name, cooling=True);


three body rate: 4



Error compiling Cython file:
------------------------------------------------------------
...
    # Initialize initial temperature
    for i in range(dims):
        data.Ts[i] = ics['T'][i]
        print("initial  temperature: %.3E" %data.Ts[i])
    
    cvdls_9species_interpolate_rates( cvdls_9species_data *data,  1)
                                     ^
------------------------------------------------------------

cvdls_9species_solver_run.pyx:802:38: 'cvdls_9species_data' is not a constant, variable or function identifier

Error compiling Cython file:
------------------------------------------------------------
...
    # Initialize initial temperature
    for i in range(dims):
        data.Ts[i] = ics['T'][i]
        print("initial  temperature: %.3E" %data.Ts[i])
    
    cvdls_9species_interpolate_rates( cvdls_9species_data *data,  1)
                                                         ^
------------------------------------------------------------

cvdls_9species_solver_run

ImportError: Building module cvdls_9species_run failed: ["CompileError: command 'gcc' failed with exit status 1\n"]

In [ ]:
temperature = 50000.0# K
density = 1.0e-1*1.00794 # cm^-3

with HidePrints():
    init, primordial = Init_values(np.array([temperature]), np.array([density]) , n_species = 9)
rv, rv_int = chemistry_run.run_cvdls_9species(init, 5.0e17,niter=1e4);

In [ ]:
flag = rv_int['successful']
plt.loglog(rv_int['t'][flag],rv_int['T'][0][flag])
rv_int['T'][0][flag][-1]

In [ ]:
total_t = 0.0
final_density = 1.0e12*1.00794
density_array = numpy.array([ init['density'] ])
pressure_array = numpy.array([])
ttt = []
run_time = []
current_density = density_array[-1]

all_data = {}
for key in init.keys():
    all_data[key] = []
all_data['force_factor'] = []

In [ ]:
dir_ff_grackle = "/home/kwoksun2/grackle/src/python/examples/freefall.h5"
import h5py
f = h5py.File(dir_ff_grackle)
fdata = f['data']
grackle_init = convert_from_grackle_to_dengo(fdata)


new_init, primordial = Init_values(np.array([temperature]), np.array([density]) , n_species = 9)
for i in new_init.keys():
    if i not in ['density','ge']:
        print(i, grackle_init[i]) 
        new_init[i] = numpy.array([grackle_init[i]])

new_init['de'] = primordial.calculate_free_electrons(new_init)
new_init['ge'] =  calculate_energy(new_init, primordial)
rv, rv_int = chemistry_run.run_cvdls_9species(new_init, 1e-5,niter=1e0)

In [ ]:
flag = rv_int['successful']
de = numpy.zeros((1))
for sp in new_init.keys():
    if sp not in ['de','density','ge','T']:
        s = species_registry[sp]
        print(s.name)
        de += rv_int[s.name][0][flag][-1]  * s.free_electrons

In [ ]:
while current_density < final_density:

    # keep track of time in here
    
    new_init = generate_init_from_results(rv_int,primordial, new_init)
    init, pressure_array, density_array, dt, force_factor = update_initial_condition(new_init, primordial, pressure_array
                                                             , density_array, safety_factor=0.01)
    tic = time.time()
    rv, rv_int = chemistry_run.run_cvdls_9species(init, dt,niter=1e4)
    toc = time.time()
    total_t += dt
    ttt.append(float(total_t))
    run_time.append(toc-tic)
    
    flag = rv_int['successful']
    for key in init.keys():
        if key not in ['density']:
            data = rv_int[key][0][flag][-1]
            all_data[key].append(data)
    all_data['force_factor'].append( float(force_factor))
    current_density = density_array[-1]
    
    t_ends = rv_int['t'][flag][-1]
    if t_ends < dt:
        print('##########cannot finish the run....####################3')
        break

In [ ]:
density_array

In [ ]:
import matplotlib.pyplot as plt
ttt = numpy.array(ttt)

In [ ]:
plt.semilogy(ttt, pressure_array,'-o')

In [ ]:
max(ttt)*u.s.in_units('yr')

In [ ]:
run_time[1000]

In [ ]:
plt.loglog(density_array[1:],run_time[:],'-o')

In [ ]:
dir_ff_grackle = "/home/kwoksun2/grackle/src/python/examples/freefall.h5"
import h5py
f = h5py.File(dir_ff_grackle)

fdata = f['data']
grackle_data = convert_from_grackle_to_dengo_all(fdata)
grackle_data['H_2']

In [ ]:
ic = 0
from matplotlib.pyplot import cm
color=cm.rainbow(np.linspace(0,1,12))

plt.figure(figsize=(10,10))
for key in all_data.keys():
    
    print(key)
    if key not in ['density','t','force_factor','ge','T']:
        data = numpy.array(all_data[key])
        data1 = grackle_data[key]
        
        plt.semilogx( density_array[:-1], data1/density_array[:-1],c=color[ic], ls = '--')
        
        plt.loglog( density_array[:-1], data/density_array[:-1],label=key, c=color[ic])
    ic += 1
plt.legend()
plt.xlabel('density (amu / $cm^{-3}$)')
plt.ylabel(r'mass density $n_x m_x / \rho$ ')

In [ ]:
plt.figure(figsize=(10,10))

H2_1 =  f['data']['H2I'] 
density = numpy.array(f['data']['density'])

data = numpy.array(all_data['H2_1']) / density_array[:-1]

plt.semilogx( density_array[1:]*u.amu_cgs, H2_1/density,label=key,c=color[0],ls='--')
plt.loglog( density_array[1:]*u.amu_cgs, 2.0*data*1.00794   ,label=key,c=color[0])
plt.xlabel('density (amu / $cm^{-3}$)')
plt.ylabel(r'mass density $n_x m_x / \rho$ ')

In [ ]:
plt.figure(figsize=(10,10))
data0 =  f['data']['temperature'] #
plt.semilogx(density, data0, label=key)

data1 = numpy.array(all_data['T']) 
plt.semilogx( density_array[1:]*u.amu_cgs, data1,label=key)

plt.xlabel('density (amu / $cm^{-3}$)')
plt.ylabel('mass density  ' + r"$n_x m_x / \rho$")

In [ ]:
plt.semilogx(density,force_factor)

In [ ]:
plt.semilogx(density,force_factor)
data = numpy.array(all_data['force_factor']) 
plt.semilogx( density_array[1:]*u.amu_cgs, data,label=key)

In [ ]:
data = numpy.array(all_data['force_factor']) 
plt.loglog( density_array[1:][data>0], data[data>0],label=key)

In [ ]:
data

In [ ]:
plt.loglog(density_array[1:], pressure_array)

In [ ]:
action.tables

In [ ]:
def cooling_rate_from_temp(T):
    cooling_rates = {}
    for action in primordial.cooling_actions.values():
        actionname = action.name
        for tab in action.tables:
            name = "{}_{}[i]".format(actionname, tab)
            
            temp_array = primordial.T
            
            cooling = numpy.interp(T, temp_array, action.tables[tab](primordial))
            cooling_rates[name] = cooling
    cooling_rates['T'] = T
    cooling_rates['z'] = numpy.zeros((len(T)))
#     cooling_rates = sorted(cooling_rates)
    return cooling_rates

def cooling_from_results(results, primordial, cooling_rates, T):
    
    cooling_results = {}

    for action in primordial.cooling_actions:
        cool_action = cooling_registry[action].equation
        cool_sym = cool_action.free_symbols
        
        cool_sym_name = [str(i) for i in cool_sym]
        func = lambdify(cool_sym, cool_action,'numpy') # returns a numpy-ready function
        
        
        num_var = len(cool_sym)
        len_T   = len(T)
        
        input_array = numpy.zeros(( num_var, len_T ))       
        for iii, sym_name in enumerate(cool_sym_name):
            try:
                input_array[iii, :] = results[sym_name]
            except:
                input_array[iii, :] = cooling_rates[sym_name]
        
        
        cooling_results[action] = func(*input_array)
    return cooling_results

In [ ]:
T_array = all_data['T']

all_data['mdensity'] = density_array[1:]
all_data['mh'] = numpy.ones((len(density_array[1:])))* u.mass_hydrogen_cgs.v

cooling_rates = cooling_rate_from_temp(T_array)


In [ ]:
cooling_results = cooling_from_results(all_data, primordial, cooling_rates, T_array)

In [ ]:
plt.figure(figsize=(10,10))
from matplotlib.pyplot import cm
color=cm.tab20(np.linspace(0,1,len(cooling_results.keys()) ))
net_cooling = numpy.zeros((len(T_array)))

iii = 0
for i in cooling_results.keys():
    d = cooling_results[i]
    
    net_cooling = d + net_cooling
    if i not in ['gammah']:
        if d[0] < 0:
            plt.semilogx(density_array[1:], numpy.abs(d)/density_array[1:]/ u.amu_cgs.v, label=i, c=color[iii])
        else:
            plt.loglog(density_array[1:], numpy.abs(d)/density_array[1:]/ u.amu_cgs.v, label=i, ls= '--', c=color[iii])
            
    iii += 1
plt.legend()
plt.xlabel('density (amu / $cm^{-3}$)')
plt.ylabel(r'cooling rate per unit mass (erg / g /s)')
plt.show()

plt.figure(figsize=(10,10))
plt.loglog(density_array[1:], numpy.abs(net_cooling)/density_array[1:]/ u.amu_cgs.v)
plt.xlabel('density (amu / $cm^{-3}$)')
plt.ylabel(r'cooling rate per unit mass (erg / g /s)')
plt.show()

In [ ]:
d = cooling_results['h2formation']
plt.loglog(density_array[1:], (d), label=i, ls= '--')

In [ ]:
from matplotlib.pyplot import cm
color=cm.rainbow(np.linspace(0,1,100))

In [ ]:
color[10]

In [ ]:
net_cooling

# TO-DO list

1. profile the code (run-time ~ 0.02s) no matter how
2. interpolate gammaH2 (it is calculate on the fly everytime)
3. include CIE cooling

In [ ]:
data/data1/1.00794

In [ ]:
sp = species_registry['H2_2']
gammaH2 = primordial.species_gamma(sp, temp=True, name=False).subs({'T':temperature})

In [ ]:
gammaH2

In [ ]:
from sympy import lambdify